In [1]:
import numpy as np

p = np.array([1e-14, 1e-14, 51555, 30555, 80e-6, 88e-6, 2e-6, 2e-6, 303.15, 1000, 885000, 723600,
0.026, 2.1e3, 1, .225200658e-5/96487, 0.8307e-11, 1.5e-6, 0.96, 0.01])

In [2]:
N1 = 5
N2 = 5

F=96487
R=8.3143
TC = 30

In [85]:
# Dp, Dn, cspmax = p[:3]
# print(Dp, Dn, cspmax)
def f(out, du, u, p, t):
    Dp = p[0]
    Dn = p[1]
    cspmax = p[2]
    csnmax = p[3]
    lp = p[4]
    lnn = p[5]
    Rpp = p[6]
    Rpn = p[7]
    T = p[8]
    ce = p[9]
    ap = p[10]
    an = p[11]
    M_sei = p[12]
    rho_sei = p[13]
    Kappa_sei = p[14]
    kp = p[15]
    kn = p[16]
    ksei = p[17]
    theta_p0 = p[18]
    theta_n0 = p[19]
    signparam = p[20]
    ratetest = p[21]
    endvolt1 = p[22]
    cc = p[23]
    

    csp = np.zeros(N1+2)
    for i in range(N1+2):
        csp[i] = u[i]
        csp[i] = max(1., min(u[i], cspmax-1))

    csn = np.zeros(N2+2)
    for i in range(N2+2):
        csn[i] = u[N1+2+i]
        csn[i] = max(1., min(u[N1+2+i], csnmax-1))
    
#     print(csp, csn)
#     print(phi_p)
    phi_p = u[N1+N2+4]
    phi_n = u[N1+N2+5]
    iint = u[N1+N2+6]
    isei = u[N1+N2+7]
    delta_sei = u[N1+N2+8]
    Q = u[N1+N2+9]
    cm = u[N1+N2+10]
    cf = u[N1+N2+11]
    pot = u[N1+N2+12]
    it = u[N1+N2+13]
    
    h1=Rpp/(N1+1)
    h2=Rpn/(N2+1)
    
    from numpy import tanh, sinh, exp
    inittime = 200
    ff = (tanh(100*((t+100)-(inittime+100)))+tanh(100*(t+100)))/2
#     ff = 1
    
    # Governing Equations
    # Positive Electrode
    out[0] = 4*csp[1] - 3*csp[0] - csp[2] # boundary condition
    for i in range(1,N1+1):
        out[i] =  Dp/h1**2 *((csp[i+1]  - csp[i-1])/i + csp[i+1] + csp[i-1] - 2*csp[i])*ff - du[i] # finite difference
    out[N1+1] = (csp[N1-1] + 3*csp[N1+1] - 4*csp[N1]) + (2*h1)*iint/F/Dp/ap/lp # second boundary condition
    
    # negative electrode
    out[N1+2] = 4*csn[1] - 3*csn[0] - csn[2]
    for i in range(1,N2+1):
        out[N1+2+i] =  Dn/h2**2 *((csn[i+1]  - csn[i-1])/i + csn[i+1] + csn[i-1] - 2*csn[i])*ff - du[N1+2+i]
    out[N1+N2+3] = (csn[N2-1] + 3*csn[N2+1] - 4*csn[N2]) - (2*h2)*iint/F/Dn/an/lnn
#     print(out)
    # so far, so good!
    
    # additional equations
    # positive electrode
    theta_p = csp[N1+1]/cspmax
    Up = (-4.656+88.669*theta_p**2-401.119*theta_p**4+342.909*theta_p**6-462.471*theta_p**8+433.434*theta_p**10)/(-1.0+18.933*theta_p**2-79.532*theta_p**4+37.311*theta_p**6-73.083*theta_p**8+95.96*theta_p**10)
    jp = 2*kp*ce**(0.5)*(cspmax-csp[N1+1])**(0.5)*csp[N1+1]**(0.5)*sinh(0.5*F/R/T*(phi_p-Up))
    out[N1+N2+4] = jp - (it/ap/F/lp)
#     print(out)
    
    #Negative electrode
    theta_n = csn[N2+1]/csnmax
    Un = 0.7222+0.1387*theta_n+0.029*theta_n**(0.5)-0.0172/theta_n+0.0019/theta_n**1.5+0.2808*exp(0.9-15*theta_n)-0.7984*exp(0.4465*theta_n-0.4108);
    jn = 2*kn*ce**(0.5)*(csnmax-csn[N2+1])**(0.5)*csn[N2+1]**(0.5)*sinh(0.5*F/R/T*(phi_n-Un+delta_sei/Kappa_sei*it/an/lnn));
    out[N1+N2+5] = jn + (iint/an/F/lnn)
    
    #C3. SEI layer Equations
    out[N1+N2+6] = -iint + it -isei
    out[N1+N2+7] = -isei+an*lnn*ksei*ce*1/(exp(F/R/T*(phi_n+delta_sei/Kappa_sei*it/an/lnn)))*signparam
    out[N1+N2+8] = isei*M_sei/F/rho_sei/an/lnn*ff - du[N1+N2+8] 
    
    #C4. Charge stored
    out[N1+N2+9] = it/3600 * ff - du[N1+N2+9];         #dQ/dt
    out[N1+N2+10] = iint/3600 * ff - du[N1+N2+10];       #dcm/dt
    out[N1+N2+11] = isei/3600 * ff - du[N1+N2+11];       #dcf/dt
    out[N1+N2+12] = pot-phi_p + phi_n;
    
    if cc == 1:
        out[N1+N2+13] = it - TC*ratetest
    else:
        out[N1+N2+13] = phi_p-phi_n-endvolt1
    
    return out

out = np.zeros(N1+N2+14)
du = np.zeros(N1+N2+14)
# u = np.zeros(N1+N2+14)
pp = np.array([1.0e-14, 1.0e-14, 51555.0, 30555.0, 8.0e-5, 8.8e-5, 2.0e-6, 2.0e-6, 303.15, 1000.0, 885000.0, 723600.0, 0.026, 2100.0, 1.0, 2.334e-11, 8.307e-12, 1.5e-6, 0.96, 0.01, 1.0, 1.0, 4.2, 1.0])


# for i in range(0, N1+N2+14):
#     u[i] = i
# f(out, du, u, pp, 0.)

differential_vars = np.ones(N1+N2+14)
differential_vars[0] = 0
differential_vars[N1+1] = 0
differential_vars[N1+2] = 0
differential_vars[N1+N2+3] = 0
differential_vars[N1+N2+4] = 0
differential_vars[N1+N2+5] = 0
differential_vars[N1+N2+6] = 0
differential_vars[N1+N2+7] = 0
differential_vars[N1+N2+12] = 0
differential_vars[N1+N2+13] = 0

u0 = np.zeros(N1+N2+14)
u0[:N1+2] = 49503.111
# u0[:N1+2] = 51003.111
# u0[1] = 49503.111
u0[(N1+2):(N1+N2+4)] = 305.55
u0[N1+N2+4] = 3.67873289259766    #phi_p
u0[N1+N2+5] = .182763748093840    #phi_n
u0[N1+N2+6] = 30                  #iint
u0[N1+N2+7] = 0                   #isei
u0[N1+N2+8] = 1e-10               #delta_sei
u0[N1+N2+9] = 0                  #Q
u0[N1+N2+10] = 0                  #cm
u0[N1+N2+11] = 0                  #cf
u0[N1+N2+12] = 3.0596914450382   #pot
u0[N1+N2+13] = TC*1   	  	  #it

out = f(out, du, u0, pp, 0.)

In [86]:
Dp = p[0]
Dn = p[1]
cspmax = p[2]
csnmax = p[3]
lp = p[4]
lnn = p[5]
Rpp = p[6]
Rpn = p[7]
T = p[8]
ce = p[9]
ap = p[10]
an = p[11]
M_sei = p[12]
rho_sei = p[13]
Kappa_sei = p[14]
kp = p[15]
kn = p[16]
ksei = p[17]
theta_p0 = p[18]
theta_n0 = p[19]
signparam = p[20]
ratetest = p[21]
endvolt1 = p[22]
cc = p[23]
N1 = p[24]
N2 = p[25]
initial = p[26]
tf = p[27]
uv[i] = p[28+]

SyntaxError: invalid syntax (<ipython-input-86-1bb85fe28957>, line 29)

In [89]:
print(list(pp))
# print(pp[23])

[1e-14, 1e-14, 51555.0, 30555.0, 8e-05, 8.8e-05, 2e-06, 2e-06, 303.15, 1000.0, 885000.0, 723600.0, 0.026, 2100.0, 1.0, 2.334e-11, 8.307e-12, 1.5e-06, 0.96, 0.01, 1.0, 1.0, 4.2, 1.0]
1.0


In [88]:
x = [4.9503e+004,4.9503e+004,4.9503e+004,4.9503e+004,4.9503e+004,4.9503e+004,4.9503e+004,3.0555e+002,3.0555e+002,3.0555e+002,3.0555e+002,3.0555e+002,3.0555e+002,3.0555e+002,3.6787e+000,1.8276e-001,3.0000e+001,0.0000e+000,1.0000e-010,0.0000e+000,0.0000e+000,0.0000e+000,3.0597e+000,3.0000e+001]
print(np.array(u0)-x)
# so, initial conditions are good.
x2 = [7.2760e-012,0.0000e+000,0.0000e+000,0.0000e+000,0.0000e+000,0.0000e+000,2.9277e+002,-5.6843e-014,0.0000e+000,0.0000e+000,0.0000e+000,0.0000e+000,0.0000e+000,-3.2552e+002,-3.6937e-005,-2.7491e+000,0.0000e+000,8.7411e-005,0.0000e+000,0.0000e+000,0.0000e+000,0.0000e+000,-4.3628e-001,0.0000e+000]
# print((out-x2)/out)
for i in range(len(x2)):
    print(x2[i], out[i], i)
# print(out[21])
# print(x2[21])


[ 1.11000000e-01  1.11000000e-01  1.11000000e-01  1.11000000e-01
  1.11000000e-01  1.11000000e-01  1.11000000e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.28925977e-05  3.74809384e-06
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -8.55496180e-06  0.00000000e+00]
7.276e-12 7.275957614183426e-12 0
0.0 0.0 1
0.0 0.0 2
0.0 0.0 3
0.0 0.0 4
0.0 0.0 5
292.77 292.7709180575773 6
-5.6843e-14 -5.684341886080802e-14 7
0.0 0.0 8
0.0 0.0 9
0.0 0.0 10
0.0 0.0 11
0.0 0.0 12
-325.52 -325.52171275058544 13
-3.6937e-05 -3.693711587696576e-05 14
-2.7491 -2.7491111154061247 15
0.0 0.0 16
8.7411e-05 8.741134951960577e-05 17
0.0 0.0 18
0.0 0.0 19
0.0 0.0 20
0.0 0.0 21
-0.43628 -0.4362776994656199 22
0.0 0.0 23


In [ ]:
# this works.
# for tomorrow:
# put into C, code up to take new initial values as well, cubic interpolation in python when changing N1, N2. Should be sick!
# Then, once that is done, remove the equations that have SEI to see just the effect of SEI?